In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#!pip install gym keras-rl2 gym[atari]

# Testing Random Enviroment in OpenAI Gym

In [4]:
import gym
import random

In [5]:
env = gym.make('SpaceInvaders-v0')
height, widht, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
print(actions)

6


In [11]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [12]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

Episode:1 score:120.0
Episode:2 score:110.0
Episode:3 score:55.0
Episode:4 score:210.0
Episode:5 score:80.0


# Creating a Deep Learngin Model with Keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (4,4), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [12]:
del model

In [13]:
model = build_model(height, widht, channels, actions)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 21, 15, 64)     65600     
_________________________________________________________________
flatten (Flatten)            (None, 60480)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               30966272  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# Build an Agent with keras

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [10]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [14]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

# Reloading Agent from memory

In [ ]:
dqn.save_weights('dqn_weights.h5f')

In [ ]:
del model, dqn

In [15]:
dqn.load_weights('dqn_weights.h5f')

In [16]:
scores = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...


C:\Users\SAMEER\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 90.000, steps: 606
